In [1]:
!pip install kafka-python psycopg2-binary


In [3]:
from kafka import KafkaConsumer
import json
import psycopg2
from datetime import datetime

In [5]:
conn = psycopg2.connect(
    dbname="kafka_orders",
    user="rohith",
    password="rohith5799",
    host="localhost",
    port="5433"
)
cursor = conn.cursor()

In [7]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS orders (
    order_id UUID PRIMARY KEY,
    customer_id INT,
    product_name VARCHAR(100),
    category VARCHAR(50),
    quantity INT,
    unit_price DECIMAL(10,2),
    total_amount DECIMAL(10,2),
    payment_method VARCHAR(50),
    shipping_country VARCHAR(50),
    timestamp TIMESTAMP
);
""")
conn.commit()

In [ ]:
consumer = KafkaConsumer(
    'orders',
    bootstrap_servers=['localhost:9092'],
    value_deserializer=lambda x: json.loads(x.decode('utf-8')),
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='order-consumer-group'
)

for message in consumer:
    order = message.value
    cursor.execute("""
        INSERT INTO orders (
            order_id, customer_id, product_name, category,
            quantity, unit_price, total_amount,
            payment_method, shipping_country, timestamp
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (order_id) DO NOTHING;
    """, (
        order['order_id'],
        order['customer_id'],
        order['product_name'],
        order['category'],
        order['quantity'],
        order['unit_price'],
        order['total_amount'],
        order['payment_method'],
        order['shipping_country'],
        datetime.fromisoformat(order['timestamp'].replace("Z", ""))
    ))

    conn.commit()
    print(f"📥 Inserted: {order['order_id']}")


📥 Inserted: c53ff6de-fa42-4142-a967-e40ee835aa9c
📥 Inserted: 748e2aff-3609-401e-9cd1-1df2a00eee51
📥 Inserted: 2282adcc-a4e1-47e0-a8c7-a63de2532ff7
📥 Inserted: fc3e02ff-135f-4630-a447-a8572a9c7168
📥 Inserted: e191dd39-0493-4d2e-82fd-218c8d575aed
📥 Inserted: 32129ae6-2305-41cd-9c83-548802a12abf
📥 Inserted: 548ef0aa-339f-4622-9238-9e1342d221e5
📥 Inserted: edb8f8af-1110-471b-969f-80675f758660
📥 Inserted: b18eb086-adcd-415d-8513-7e6fd81c7439
📥 Inserted: 9c5506bd-d094-4267-b977-7a750d885b38
📥 Inserted: b15b0aa1-db2a-4eba-a321-cf0ed862b2aa
📥 Inserted: 9bd3717f-c4bd-4210-8163-3159a8d9b702
📥 Inserted: 5368aa40-b421-4260-b30c-0aea68d60fb9
📥 Inserted: 23d0c466-5d9c-401c-bf33-63ea91f6d4b4
📥 Inserted: 22cd5281-c34b-4c3a-b32e-68d7ca26164f
📥 Inserted: 552b3983-c253-439b-b1a4-ab495a3f7182
📥 Inserted: 9ddf54ae-eecd-47f2-9313-6554f4ede9a0
📥 Inserted: 0f2ba0a6-c76b-4e5e-bfa2-1f6cf8796b1b
📥 Inserted: 1b98de81-0699-4fd9-a7d7-94c5591279e8
📥 Inserted: cc731745-ecb3-47f7-940c-afb1801a720e
📥 Inserted: 51546bcd